Data Source

https://www.kaggle.com/datasets/luisblanche/covidct

In [ ]:
# pip install tensorflow==2.13.0

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

2024-10-30 16:04:29.548474: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 16:04:29.583270: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-30 16:04:30.291798: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.executing_eagerly()

True

In [ ]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
# import tensorflow.keras.backend as K
#tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)


In [ ]:
import tensorflow as tf
def limitgpu(maxmem):
	gpus = tf.config.list_physical_devices('GPU')
	if gpus:
		# Restrict TensorFlow to only allocate a fraction of GPU memory
		try:
			for gpu in gpus:
				tf.config.experimental.set_virtual_device_configuration(gpu,
						[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=maxmem)])
		except RuntimeError as e:
			# Virtual devices must be set before GPUs have been initialized
			print(e)


# 1.5GB
limitgpu(1024+512) 


In [ ]:
pip show tensorflow

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Create 2 virtual GPUs with 1GB memory each
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024),
         tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:
!pip install opencv-python==3.4.18.65

In [ ]:
## Importing Relevant Libraries

In [3]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
a = tf.zeros([], tf.float32)
## Imports libs

2024-10-30 16:04:37.918455: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:227] Using CUDA malloc Async allocator for GPU: 0
2024-10-30 16:04:37.918552: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8783 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2024-10-30 16:04:37.919214: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:227] Using CUDA malloc Async allocator for GPU: 1
2024-10-30 16:04:37.919263: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14457 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2024-10-30 16:04:37.919797: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:227] Using CUDA malloc Async allocator for GPU: 2
2024-10-30 16:04:37.919836: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639

In [ ]:
print("OpenCV version:", cv2.__version__)

In [ ]:
pip install lime==0.1.1.29

In [ ]:
pip show lime

## Renaming images data to Specified format

In [4]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
!pwd

In [5]:
path1 = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_merged/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_merged/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

## Data Preprocessing
### Processing images for machine learning to create two-class dataset

In [6]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

#### Processing CT_scan Covid-19  CT_scan nonCovid-19 images

In [7]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covnid-19 negative

In [ ]:
ct_covid_features_df

In [ ]:
#imshow(ct_covid_features_df.iloc[0].to_numpy().reshape((100,150)))

In [ ]:
ct_noncovid_features_df

In [ ]:
#imshow(ct_noncovid_features_df.iloc[1].to_numpy().reshape((100,150)))

#### Concatenating the two data frame to create a dataset ready for machine learning

In [8]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

,0,1,2,3,4,5,6,7,8,9,...,89991,89992,89993,89994,89995,89996,89997,89998,89999,class_label
Non-Covid (14).png,64,64,64,64,63,63,63,63,63,63,...,63,63,63,62,63,63,65,67,68,0
Non-Covid (227).png,63,63,64,64,65,65,65,65,65,66,...,210,211,224,231,239,233,185,144,112,0
Non-COVID-19_757.png,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Non-COVID-19_714.png,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Non-COVID-19_504.png,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
COVID-19_0432.png,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Covid (254).png,193,193,197,194,198,198,196,196,197,195,...,255,255,255,255,255,255,255,255,255,1
Non-COVID-19_046.png,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Non-Covid (557).png,61,61,61,64,63,63,63,63,63,63,...,210,209,208,211,216,222,229,232,232,0


In [ ]:
ct_covid_features_df.shape

In [ ]:
ct_noncovid_features_df.shape

In [ ]:
cvd_imgs_dataset.shape

In [ ]:
#### Prepare negative covid images for machine learning ready

In [9]:
input_data_x_ = ct_noncovid_features_df.iloc[:,:-1].to_numpy().reshape((2173,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df['class_label'])
ct_noncovid_features_df
ct_noncovid_features_df = pd.get_dummies(ct_noncovid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_ = np.array(ct_noncovid_features_df[['output_encode_0']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x_.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_.shape))

Input_x Data Shape: 
(2173, 300, 300, 1)
Output_y Data Shape: 
(2173, 1)


In [10]:
#### Prepare positive covid images for machine learning ready

In [11]:
input_data_x__ = ct_covid_features_df.iloc[:,:-1].to_numpy().reshape((2476,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df['output_encode'] = label_encoder.fit_transform(ct_covid_features_df['class_label'])
ct_covid_features_df
ct_covid_features_df = pd.get_dummies(ct_covid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__ = np.array(ct_covid_features_df[['output_encode_0']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x__.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y__.shape))

Input_x Data Shape: 
(2476, 300, 300, 1)
Output_y Data Shape: 
(2476, 1)


In [12]:
input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((4649,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))

Input_x Data Shape: 
(4649, 300, 300, 1)
Output_y Data Shape: 
(4649, 2)


In [ ]:
type(input_data_x)

In [ ]:
input_data_x.shape

In [ ]:
output_label_y.shape

In [ ]:
## Images of Negative Covid-19
for i in range(10):
    imshow(input_data_x_[i])
    show()
    

In [ ]:
## Images of Positive Covid-19
for i in range(15):
    imshow(input_data_x__[i])
    show()
    

#### visualize first 5 images from the dataset

In [ ]:
for i in range(15):
    imshow(input_data_x[i])
    show()
    

### Feature Scaling

#### **Splitting the dataset into the Training set and Test set**

In [13]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    input_data_x, output_label_y, test_size=.20, random_state=42)

In [14]:
from sklearn.model_selection import train_test_split

train_features_, test_features_, train_labels_, test_labels_ = train_test_split(
    input_data_x_, output_label_y_, test_size=.20, random_state=42)

In [15]:
from sklearn.model_selection import train_test_split

train_features__, test_features__, train_labels__, test_labels__ = train_test_split(
    input_data_x__, output_label_y__, test_size=.20, random_state=42)

In [ ]:
train_features__.shape

In [ ]:
test_features__.shape

In [16]:
from tensorflow.keras.layers import Input, MaxPooling3D, AveragePooling3D, Dense, Dropout, GlobalAveragePooling3D
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# Define the conv3d_bn function as above
def conv3d_bn(x, filters, num_row, num_col, num_depth, strides=(1,1,1), padding='same', name=None):
    x = layers.Conv3D(
        filters, 
        kernel_size=(num_depth, num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=name+'_conv3d')(x)
    x = layers.BatchNormalization(axis=-1, scale=False, name=name+'_bn')(x)
    x = layers.Activation('relu', name=name+'_relu')(x)
    return x

# Define the Inception_Inflated3d function as updated above
def Inception_Inflated3d(include_top=True,
                         weights=None,
                         input_tensor=None,
                         input_shape=None,
                         dropout_prob=0.0,
                         endpoint_logit=True,
                         classes=400):
    
    img_input = Input(shape=input_shape)
    channel_axis = -1  # channels_last

    x = conv3d_bn(img_input, 64, 7, 7, 7, strides=(2, 2, 2), padding='same', name='Conv3d_1a_7x7')
    # Downsampling (spatial only)
    x = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same', name='MaxPool2d_2a_3x3')(x)
    x = conv3d_bn(x, 64, 1, 1, 1, strides=(1, 1, 1), padding='same', name='Conv3d_2b_1x1')
    x = conv3d_bn(x, 192, 3, 3, 3, strides=(1, 1, 1), padding='same', name='Conv3d_2c_3x3')
    # Downsampling (spatial only)
    x = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same', name='MaxPool2d_3a_3x3')(x)
    # Mixed 3b
    branch_0 = conv3d_bn(x, 64, 1, 1, 1, padding='same', name='Conv3d_3b_0a_1x1')
    branch_1 = conv3d_bn(x, 96, 1, 1, 1, padding='same', name='Conv3d_3b_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 128, 3, 3, 3, padding='same', name='Conv3d_3b_1b_3x3')
    branch_2 = conv3d_bn(x, 16, 1, 1, 1, padding='same', name='Conv3d_3b_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 32, 3, 3, 3, padding='same', name='Conv3d_3b_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_3b_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 32, 1, 1, 1, padding='same', name='Conv3d_3b_3b_1x1')
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_3b')
    # Mixed 3c
    branch_0 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_3c_0a_1x1')
    branch_1 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_3c_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 192, 3, 3, 3, padding='same', name='Conv3d_3c_1b_3x3')
    branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_3c_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 96, 3, 3, 3, padding='same', name='Conv3d_3c_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_3c_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_3c_3b_1x1')
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_3c')
    # Downsampling (spatial and temporal)
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2), padding='same', name='MaxPool2d_4a_3x3')(x)
    # Mixed 4b
    branch_0 = conv3d_bn(x, 192, 1, 1, 1, padding='same', name='Conv3d_4b_0a_1x1')
    branch_1 = conv3d_bn(x, 96, 1, 1, 1, padding='same', name='Conv3d_4b_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 208, 3, 3, 3, padding='same', name='Conv3d_4b_1b_3x3')
    branch_2 = conv3d_bn(x, 16, 1, 1, 1, padding='same', name='Conv3d_4b_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 48, 3, 3, 3, padding='same', name='Conv3d_4b_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4b_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4b_3b_1x1')
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4b')
    # Mixed 4c
    branch_0 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_4c_0a_1x1')
    branch_1 = conv3d_bn(x, 112, 1, 1, 1, padding='same', name='Conv3d_4c_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 224, 3, 3, 3, padding='same', name='Conv3d_4c_1b_3x3')
    branch_2 = conv3d_bn(x, 24, 1, 1, 1, padding='same', name='Conv3d_4c_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4c_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4c_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4c_3b_1x1')
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4c')
    # Mixed 4d
    branch_0 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_4d_0a_1x1')
    branch_1 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_4d_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 256, 3, 3, 3, padding='same', name='Conv3d_4d_1b_3x3')
    branch_2 = conv3d_bn(x, 24, 1, 1, 1, padding='same', name='Conv3d_4d_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4d_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4d_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4d_3b_1x1')
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4d')
    # Mixed 4e
    branch_0 = conv3d_bn(x, 112, 1, 1, 1, padding='same', name='Conv3d_4e_0a_1x1')
    branch_1 = conv3d_bn(x, 144, 1, 1, 1, padding='same', name='Conv3d_4e_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 288, 3, 3, 3, padding='same', name='Conv3d_4e_1b_3x3')
    branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_4e_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4e_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4e_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4e_3b_1x1')
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4e')
    # Mixed 4f
    branch_0 = conv3d_bn(x, 256, 1, 1, 1, padding='same', name='Conv3d_4f_0a_1x1')
    branch_1 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_4f_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 320, 3, 3, 3, padding='same', name='Conv3d_4f_1b_3x3')
    branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_4f_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_4f_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4f_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_4f_3b_1x1')
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4f')
    # Downsampling (spatial and temporal)
    x = MaxPooling3D((2, 2, 2), strides=(2, 2, 2), padding='same', name='MaxPool2d_5a_2x2')(x)
    # Mixed 5b
    branch_0 = conv3d_bn(x, 256, 1, 1, 1, padding='same', name='Conv3d_5b_0a_1x1')
    branch_1 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_5b_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 320, 3, 3, 3, padding='same', name='Conv3d_5b_1b_3x3')
    branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_5b_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_5b_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_5b_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_5b_3b_1x1')
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_5b')
    # Mixed 5c
    branch_0 = conv3d_bn(x, 384, 1, 1, 1, padding='same', name='Conv3d_5c_0a_1x1')
    branch_1 = conv3d_bn(x, 192, 1, 1, 1, padding='same', name='Conv3d_5c_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 384, 3, 3, 3, padding='same', name='Conv3d_5c_1b_3x3')
    branch_2 = conv3d_bn(x, 48, 1, 1, 1, padding='same', name='Conv3d_5c_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_5c_2b_3x3')
    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_5c_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_5c_3b_1x1')
    
    if include_top:
        x = GlobalAveragePooling3D(name='global_avg_pool')(x)
        if dropout_prob > 0:
            x = Dropout(dropout_prob, name='dropout')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    
    else:
        x = GlobalAveragePooling3D(name='global_avg_pool')(x)
    
    model = Model(img_input, x, name='i3d_inception')
    return model


In [17]:
train_features_Incep = np.repeat(train_features[:, np.newaxis, :, :, :], 10, axis=1)
train_features_Incep.shape

(3719, 10, 300, 300, 1)

In [18]:
model = Inception_Inflated3d(include_top=True, input_shape=(10, 300, 300, 1), classes=2)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_features_Incep, train_labels, epochs=10, batch_size=8)

Epoch 1/10


2024-10-30 16:07:15.418421: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2024-10-30 16:07:15.906105: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-30 16:07:16.995218: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3ce2bf60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-30 16:07:16.995275: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2024-10-30 16:07:16.995291: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Quadro RTX 5000, Compute Capability 7.5
2024-10-30 16:07:16.995303: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): Quadro RTX 5000, Compute Capability 7.5
2024-10-30 16:07:16.995314: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): Quad

465/465 [==============================] - 155s 293ms/step - loss: 0.5056 - accuracy: 0.7642
Epoch 2/10
465/465 [==============================] - 134s 289ms/step - loss: 0.4285 - accuracy: 0.8099
Epoch 3/10
465/465 [==============================] - 134s 288ms/step - loss: 0.3998 - accuracy: 0.8223
Epoch 4/10
465/465 [==============================] - 134s 288ms/step - loss: 0.3468 - accuracy: 0.8486
Epoch 5/10
465/465 [==============================] - 134s 288ms/step - loss: 0.3324 - accuracy: 0.8658
Epoch 6/10
465/465 [==============================] - 134s 288ms/step - loss: 0.3387 - accuracy: 0.8564
Epoch 7/10
465/465 [==============================] - 134s 288ms/step - loss: 0.3313 - accuracy: 0.8604
Epoch 8/10
465/465 [==============================] - 134s 288ms/step - loss: 0.2922 - accuracy: 0.8849
Epoch 9/10
465/465 [==============================] - 134s 288ms/step - loss: 0.2633 - accuracy: 0.8935
Epoch 10/10
465/465 [==============================] - 134s 288ms/step - lo

In [ ]:
model.save('inception_model.h5')

### IMPLEMENTATION OF SHAP

In [ ]:
import shap
from shap import initjs
initjs()
custom_objects = None
model_mse = tf.keras.models.load_model('cvd_cnn_1.h5', custom_objects=custom_objects, compile=False)

In [ ]:
model_mse.summary()

In [ ]:
masked_images_as_lists = []
image_shape = (300, 300, 1)

for image in test_features:
    # Ensure the image has the correct shape
    if image.shape != image_shape:
        raise ValueError(f"Image shape {image.shape} does not match expected shape {image_shape}")

    # Initialize the masker for a single image
    masker = shap.maskers.Image("blur(32,32)", image_shape)

    # Apply the masker to each image
    masked_image_tuple = masker(image, np.zeros(image.shape[:-1]))  # Adjust the mask as needed
    
    # Extract the first element of the tuple (which is the masked image)
    masked_image = masked_image_tuple[0]
    
    # Convert the masked image to a list and store it
    masked_images_as_lists.append(masked_image.tolist())


In [ ]:
len(masked_images_as_lists)

In [ ]:
len(test_features)

In [ ]:
### Explainer for our both negative and positive images

In [ ]:
explain = shap.GradientExplainer(model_mse,test_features)
explain

In [ ]:
### Explainer for our positive images

In [ ]:
explain1 = shap.GradientExplainer(model_mse,test_features__)
explain1

In [ ]:
### Explainer for our negative images

In [ ]:
explain0 = shap.GradientExplainer(model_mse,test_features_)
explain0

In [ ]:
### Check for the Images on the test features both positive and negative

In [ ]:
for i in range(15):
    imshow(test_features[i])
    show()

In [ ]:
### Compute shap values for both positive and negative test sample

In [ ]:
shap_values = explain.shap_values(test_features)

In [ ]:
shap_values.shape

In [ ]:
## Compute shap values for positive test sample

In [ ]:
shap_values1 = explain1.shap_values(test_features__)

In [ ]:
shap_values1.shape

In [ ]:
## Compute shap values for negative test sample

In [ ]:
shap_values0 = explain1.shap_values(test_features_)

In [ ]:
shap_values0.shape

In [ ]:
shap.image_plot(shap_values0[:5], test_features_[:5])

In [ ]:
### Plot for both postive and negative test sammple

In [ ]:
shap.image_plot(shap_values[:5], test_features[:5])

In [ ]:
### Plot for positive test sample

In [ ]:
shap.image_plot(shap_values0[:5], test_features__[:5])

In [ ]:
shap.image_plot(my_shap_values, my_features,aspect=0.9, hspace=0.2)

In [ ]:
###Explainer for Inception Model

In [ ]:
model_incept = tf.keras.models.load_model('inception_model.h5')

In [ ]:
model_incept.summary()

In [ ]:
import shap

In [ ]:
explainInception = shap.GradientExplainer(model_incept)
explainInception 

In [ ]:
shap_values_incept = explainInception.shap_values(test_features_Incep)

In [ ]:
shap.image_plot(shap_values_incept[:5], test_features[:5])

In [ ]:
##### Remove non important regions

In [ ]:
my_new_shap = shap_values_incept[:, :, :, :, 0]
my_new_shap.shape

In [ ]:
threshold = np.percentile(np.abs(shap_values), 95)
save_path = "dataset_important_features/Inception/SHAP" 
process_and_save_images(test_features, my_new_shap, threshold, save_path)

In [ ]:
### Retrain on important features

In [ ]:
new_path = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/Inception/Shap/'
new_prefix = 'ct_covid'

In [ ]:
ct_covid_shap_features_Incept_df =  processImages(new_path,1)
ct_covid_shap_features_Incept_df

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_shap_features_Incept_df]
cvd_imgs_incept_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap_incept = cvd_imgs_incept_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap_incept)

In [ ]:
input_data_shap_incept_x = cvd_imgs_dataset_shap_incept.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap_incept['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap_incept['class_label'])
cvd_imgs_dataset_shap_incept
cvd_imgs_dataset_shap_incept = pd.get_dummies(cvd_imgs_dataset_shap, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_shap_incept_y = np.array(cvd_imgs_dataset_shap[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_shap_incept_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_shap_incept_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_incept_shap, test_features_incept_shap, train_labels_incept_shap, test_labels_incept_shap = train_test_split(
    input_data_shap_incept_x, output_label_shap_incept_y, test_size=.20, random_state=42)

In [ ]:
model_incept_import = Inception_Inflated3d(include_top=True, input_shape=(10, 300, 300, 1), classes=2)

# Compile the model
model_incept_import.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model_incept_import.fit(train_features_incept_shap, train_labels_incept_shap, epochs=10, batch_size=8)

In [ ]:
#### Apply LIME 

In [ ]:
train_features_Incep[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_incept_import.predict(images)

single_image = train_features_Incep[0] 
single_image.shape[0]
num_images_to_explain = 300
explainer = lime_image.LimeImageExplainer()
explanations = []
masks = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image = test_features[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    
    # Get explanation for the top predicted class
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks.append(mask)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
masks_incept = np.array(masks)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imsave
from skimage.util import img_as_ubyte
from skimage.color import rgb2gray
from skimage.filters import gaussian
import os

def remove_non_important_features(images, masks, method='mean', blur_sigma=5, save=False, save_path='dataset_important_features'):
    if not isinstance(images, (list, np.ndarray)):
        raise TypeError("Images should be a list or numpy array of image arrays.")
        
    if not isinstance(masks, (list, np.ndarray)):
        raise TypeError("Masks should be a list or numpy array of mask arrays.")
        
    if len(images) != len(masks):
        raise ValueError("The number of images and masks must be the same.")
    
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)
    
    processed_images = []
    
    for idx, (image, mask) in enumerate(zip(images, masks)):
        if image.shape[:2] != mask.shape:
            raise ValueError(f"Image and mask at index {idx} have mismatched dimensions.")
        
        processed_image = image.copy()
        
        if method == 'mean':
            if image.ndim == 3:  # Color image
                mean_value = image.mean(axis=(0,1), keepdims=True)
            else:  # Grayscale image
                mean_value = image.mean()
            processed_image[mask == 0] = mean_value
        
        elif method == 'zero':
            processed_image[mask == 0] = 0
        
        elif method == 'blur':
            blurred_image = gaussian(processed_image, sigma=blur_sigma, multichannel=True)
            processed_image[mask == 0] = blurred_image[mask == 0]
        
        else:
            raise ValueError("Invalid method chosen. Available methods: 'mean', 'zero', 'blur'.")
        
        processed_images.append(processed_image)
        
        if save:
            image_to_save = img_as_ubyte(processed_image)
            imsave(os.path.join(save_path, f'processed_image_{idx}.png'), image_to_save)
            print(f"Saved: {os.path.join(save_path, f'processed_image_{idx}.png')}")
    
    return processed_images


In [ ]:
from skimage.transform import resize

def resize_mask(mask, image_shape):
    return resize(mask, image_shape[:2], mode='reflect', anti_aliasing=True, preserve_range=True)


In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image[0].shape}")
print(f"Original Mask shape: {masks[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask, image.shape) for mask, image in zip(masks, single_image)]

# Process images with the resized masks
processed_images = remove_non_important_features(
    images=single_image,
    masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/Inception/LIME'  # Directory to save images
)


In [ ]:
## Train on new dataset

In [ ]:
new_path_ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/Inception/LIME/'
new_prefix_ = 'ct_covid'

In [ ]:
ct_covid_Lime_features_Incept_df =  processImages(new_path,1)
ct_covid_Lime_features_Incept_df

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_Lime_features_Incept_df]
cvd_imgs_incept_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_lime_incept = cvd_imgs_incept_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_lime_incept)

In [ ]:
input_data_lime_incept_x = cvd_imgs_dataset_lime_incept.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_lime_incept['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_lime_incept['class_label'])
cvd_imgs_dataset_lime_incept
cvd_imgs_dataset_lime_incept = pd.get_dummies(cvd_imgs_dataset_lime_incept, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_lime_incept_y = np.array(cvd_imgs_dataset_lime_incept[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_lime_incept_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_lime_incept_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_incept_lime, test_features_incept_lime, train_labels_incept_lime, test_labels_incept_lime = train_test_split(
    input_data_lime_incept_x, output_label_lime_incept_y, test_size=.20, random_state=42)

In [ ]:
model_incept_import = Inception_Inflated3d(include_top=True, input_shape=(10, 300, 300, 1), classes=2)

# Compile the model
model_incept_import.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model_incept_import.fit(train_features_incept_lime, train_labels_incept_lime, epochs=10, batch_size=8)

In [ ]:
### Apply Interception

In [ ]:
masker_shap = np.array(masked_images_as_lists)

In [ ]:
masker_shap_ = masker_shap[:, 0, :, :]
len(masker_shap_[:300])

In [ ]:
test_features_ = test_features[:, :, :, 0]
len(test_features_[:300])

In [ ]:
import numpy as np
import os
from skimage.io import imsave
from skimage.util import img_as_ubyte

def find_intersection_and_save(images, shap_masks, lime_masks, save=False, save_path='dataset_intersection'):
    """
    Finds the intersection between SHAP and LIME masks and applies it to images.
    Highlights the intersection regions and overlays them on the original image.

    Args:
        images (list or np.ndarray): List or array of images. Each image should be a numpy array.
        shap_masks (list or np.ndarray): SHAP binary masks for important features.
        lime_masks (list or np.ndarray): LIME binary masks for important features.
        save (bool): Whether to save the images with important features.
        save_path (str): Directory path to save the images if save is True.

    Returns:
        intersection_images (list): List of images with intersection regions highlighted.
    """
    if len(images) != len(shap_masks) or len(images) != len(lime_masks):
        raise ValueError("The number of images, SHAP masks, and LIME masks must be the same.")
    
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)

    intersection_images = []

    for idx, (image, shap_mask, lime_mask) in enumerate(zip(images, shap_masks, lime_masks)):
        # Ensure image and masks have matching dimensions
        if image.shape[:2] != shap_mask.shape or image.shape[:2] != lime_mask.shape:
            raise ValueError(f"Image and masks at index {idx} have mismatched dimensions.")
        
        # Ensure masks are binary and of boolean type
        shap_mask = shap_mask.astype(bool)
        lime_mask = lime_mask.astype(bool)
        
        # Find the intersection of SHAP and LIME masks
        intersection_mask = np.logical_and(shap_mask, lime_mask)
        
        # Create a copy of the image to process
        intersection_image = np.copy(image)

        # Ensure the image is in RGB format for highlighting
        if intersection_image.ndim == 2:  # For grayscale images
            intersection_image = np.stack([intersection_image] * 3, axis=-1)  # Convert to RGB

        # Create an overlay image with the highlighted regions
        overlay_image = np.copy(intersection_image)
        highlight_color = [255, 0, 0]  # Red color

        # Apply the highlight color to the intersection areas
        overlay_image[intersection_mask] = highlight_color

        # Combine original image and overlay image
        final_image = np.where(intersection_mask[:, :, np.newaxis], overlay_image, intersection_image)

        intersection_images.append(final_image)
        
        if save:
            # Convert image to uint8 before saving
            image_to_save = img_as_ubyte(final_image)

            # Ensure image is either 2D or 3D
            if len(image_to_save.shape) not in [2, 3]:
                raise ValueError(f"Unexpected image shape for saving: {image_to_save.shape}")

            # Save the image
            imsave(os.path.join(save_path, f'intersection_image_{idx}.png'), image_to_save)
    
    return intersection_images


In [ ]:
intersection_images = find_intersection_and_save(test_features_[:300], masker_shap_[:300], train_features_incept_lime, save=True, save_path="/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/inception/intersection/")

In [ ]:
### Train on new dataset

In [ ]:
new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/inception/intersection/'
new_prefix_ = 'ct_covid'

In [ ]:
def processImages(imgDirPath,binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_intersection_features_df =  processImages(new_path__,1)
ct_covid_intersection_features_df.shape

In [ ]:
cvd_imgs_intersection = [ct_noncovid_features_df, ct_covid_intersection_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs_intersection)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_intersection = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_intersection)

In [ ]:
input_data_intersection_x = cvd_imgs_dataset_intersection.iloc[:,:-1].to_numpy().reshape((2473,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_intersection['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_intersection['class_label'])
cvd_imgs_dataset_intersection
cvd_imgs_dataset_intersection = pd.get_dummies(cvd_imgs_dataset_intersection, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_intersection_y = np.array(cvd_imgs_dataset_intersection[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_intersection_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_intersection_y.shape))

In [ ]:
for i in range(3):
    imshow(input_data_intersection_x[i])
    show()

In [ ]:
from sklearn.model_selection import train_test_split

train_features_intersection, test_features_intersection, train_labels_intersection, test_labels_intersection = train_test_split(
    input_data_intersection_x, output_label_intersection_y, test_size=.20, random_state=42)

In [ ]:
model_incept_import = Inception_Inflated3d(include_top=True, input_shape=(10, 300, 300, 1), classes=2)

# Compile the model
model_incept_import.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model_incept_import.fit(train_features_intersection, train_labels_intersection, epochs=10, batch_size=8)

In [ ]:
###VGG19

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense

In [ ]:
def VGG19(input_shape=None, classes=5, use_soft=True):
    img_input = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(2, (3, 3), activation='relu', padding='same', name='block1_conv1', kernel_initializer="he_normal")(img_input)
    x = layers.Conv2D(2, (3, 3), activation='relu', padding='same', name='block1_conv2', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(4, (3, 3), activation='relu', padding='same', name='block2_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(4, (3, 3), activation='relu', padding='same', name='block2_conv2', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Classification block
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dense(512, activation='relu', name='fc1')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu', name='fc2')(x)

    # Output layer
    if use_soft:
        x = Dense(classes, activation="softmax", name='predictions')(x)
    else:
        x = Dense(classes, activation="linear", name="Z_4")(x)

    modelss = models.Model(img_input, x, name='vgg19')

    return modelss


In [ ]:
modelss = VGG19(input_shape=(300, 300, 1), classes=2)
modelss.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
train_features.shape

In [ ]:
train_labels.shape

In [ ]:
modelss.fit(train_features, train_labels, epochs=20, validation_data=(test_features, test_labels))

In [ ]:
modelss.save('VGG19_model.h5')

In [ ]:
###Explainer for VGG19 Model

In [ ]:
model_VGG19 = tf.keras.models.load_model('VGG19_model.h5')

In [ ]:
model_VGG19.summary()

In [ ]:
explainVGG19 = shap.GradientExplainer(model_VGG19,train_features)
explainVGG19 

In [ ]:
shap_values_ = explainVGG19.shap_values(test_features)

In [ ]:
shap_values_.shape

In [ ]:
shap.image_plot(shap_values_[:5], test_features[:5])

In [ ]:
###Mask out the Important regions

In [ ]:
import numpy as np
import cv2
import os

import numpy as np
import cv2
import os

def process_and_save_images(images, shap_values, threshold, output_folder):
    """
    Process images by removing non-important features based on SHAP values,
    change the background to white, and apply a red color to non-important features.

    Parameters:
    - images: numpy array of shape (num_images, height, width, channels)
    - shap_values: numpy array of shape (num_images, height, width, channels)
    - threshold: importance threshold to decide which features are important
    - output_folder: directory to save the processed images
    """

    # Create masks based on SHAP values and threshold
    important_features_mask = np.abs(shap_values) > threshold

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process and save each image
    for idx, (img, mask) in enumerate(zip(images, important_features_mask)):
        # Ensure image and mask are of the same shape
        if img.shape != mask.shape:
            raise ValueError(f"Image shape {img.shape} and mask shape {mask.shape} do not match.")
        
        # Create a white background
        white_background = np.ones_like(img) * 255

        # Apply mask to keep important features and set the rest to white
        processed_img = np.where(mask, img, white_background)

        # Convert non-important features (where mask is False) to red (for RGB images)
        if img.shape[-1] == 3:  # If the image has 3 channels (colored image)
            red_mask = np.zeros_like(img)
            red_mask[..., 2] = 255  # Red color in RGB

            # Ensure to only modify non-important features (where mask is False)
            processed_img[~mask] = red_mask[~mask]

        # Ensure the image is within the range 0-255 for saving
        processed_img = processed_img.astype(np.uint8)

        # Save the image
        output_file = os.path.join(output_folder, f'processed_image_{idx}.png')
        cv2.imwrite(output_file, processed_img)


In [ ]:
my_model_shap = shap_values_[:, :, :, :, 0]
my_model_shap.shape

In [ ]:
threshold = np.percentile(np.abs(shap_values_), 95)
save_path = "dataset_important_features/VGG19/SHAP"  # Replace with your directory path
process_and_save_images(test_features, my_model_shap, threshold, save_path)

In [ ]:
###Retrain the model on important images only

In [ ]:
new_path = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/SHAP/'
new_prefix = 'ct_covid'

In [ ]:
def processImages(imgDirPath,binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_shap_features_VGG19_df =  processImages(new_path,1)
ct_covid_shap_features_VGG19_df

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_shap_features_VGG19_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap_VGG19 = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap_VGG19)

In [ ]:
cvd_imgs_dataset_shap_VGG19.shape

In [ ]:
input_data_shap_VGG19_x = cvd_imgs_dataset_shap_VGG19.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap_VGG19['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap_VGG19['class_label'])
cvd_imgs_dataset_shap_VGG19
cvd_imgs_dataset_shap_VGG19 = pd.get_dummies(cvd_imgs_dataset_shap_VGG19, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_shap_VGG19_y = np.array(cvd_imgs_dataset_shap_VGG19[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_shap_VGG19_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_shap_VGG19_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap_VGG19, test_features_shap_VGG19, train_labels_shap_VGG19, test_labels_shap_VGG19 = train_test_split(
    input_data_shap_VGG19_x, output_label_shap_VGG19_y, test_size=.20, random_state=42)

In [ ]:
train_features_shap_VGG19.shape

In [ ]:
from keras import models
from keras.models import *
from keras.layers import *
from tensorflow.keras.optimizers import Adam


def VGG19(input_shape=None, classes=5, use_soft=True):
    img_input = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(2, (3, 3), activation='relu', padding='same', name='block1_conv1', kernel_initializer="he_normal")(img_input)
    x = layers.Conv2D(2, (3, 3), activation='relu', padding='same', name='block1_conv2', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(4, (3, 3), activation='relu', padding='same', name='block2_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(4, (3, 3), activation='relu', padding='same', name='block2_conv2', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Classification block
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dense(512, activation='relu', name='fc1')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu', name='fc2')(x)

    # Output layer
    if use_soft:
        x = Dense(classes, activation="softmax", name='predictions')(x)
    else:
        x = Dense(classes, activation="linear", name="Z_4")(x)

    model_vgg19 = models.Model(img_input, x, name='vgg19')

    return model_vgg19


In [ ]:
model_vgg19 = VGG19(input_shape=(300, 300, 1), classes=2)
model_vgg19.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_vgg19.fit(train_features_shap_VGG19, train_labels_shap_VGG19, epochs=10, validation_data=(test_features_shap_VGG19, test_labels_shap_VGG19))

In [ ]:
###Apply LIME

In [ ]:
train_features[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_vgg19.predict(images)

single_image = train_features_Incep[0] 
single_image.shape[0]
num_images_to_explain = 300
explainer = lime_image.LimeImageExplainer()
explanations = []
masks = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image = test_features[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    
    # Get explanation for the top predicted class
    temp, mask_VGG16 = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks.append(mask)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
masks_VGG19 = np.array(masks)

In [ ]:
from skimage.transform import resize

def resize_mask(mask, image_shape):
    return resize(mask, image_shape[:2], mode='reflect', anti_aliasing=True, preserve_range=True)


In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image[0].shape}")
print(f"Original Mask shape: {masks[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask, image.shape) for mask, image in zip(masks, single_image)]

# Process images with the resized masks
processed_images = remove_non_important_features(
    images=single_image,
    masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/VGG19/LIME'  # Directory to save images
)


In [ ]:
### Train on new dataset

In [ ]:
new_path_ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/LIME/'
new_prefix_ = 'ct_covid'

In [ ]:
ct_covid_Lime_features_VGG19_df =  processImages(new_path,1)
ct_covid_Lime_features_VGG19_df

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_Lime_features_VGG19_df]
cvd_imgs_VGG19_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_lime_VGG19 = cvd_imgs_VGG19_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_lime_VGG19)

In [ ]:
input_data_lime_VGG19_x = cvd_imgs_dataset_lime_VGG19.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_lime_VGG19['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_lime_VGG19['class_label'])
cvd_imgs_dataset_lime_VGG19
cvd_imgs_dataset_lime_VGG19 = pd.get_dummies(cvd_imgs_dataset_lime_VGG19, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_lime_VGG19_y = np.array(cvd_imgs_dataset_lime_VGG19[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_lime_VGG19_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_lime_VGG19_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_VGG19_lime, test_features_VGG19_lime, train_labels_VGG19_lime, test_labels_VGG19_lime = train_test_split(
    input_data_lime_VGG19_x, output_label_lime_VGG19_y, test_size=.20, random_state=42)

In [ ]:
model_vgg19 = VGG19(input_shape=(300, 300, 1), classes=2)
model_vgg19.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_vgg19.fit(train_features_VGG19_lime, train_labels_VGG19_lime, epochs=10, validation_data=(test_features_VGG19_lime, test_labels_VGG19_lime))

In [ ]:
### Intersection

In [ ]:
masker_shap = np.array(masked_images_as_lists)

In [ ]:
masker_shap_ = masker_shap[:, 0, :, :]
len(masker_shap_[:300])

In [ ]:
test_features_ = test_features[:, :, :, 0]
len(test_features_[:300])

In [ ]:
import numpy as np
import os
from skimage.io import imsave
from skimage.util import img_as_ubyte

def find_intersection_and_save(images, shap_masks, lime_masks, save=False, save_path='dataset_intersection'):
    """
    Finds the intersection between SHAP and LIME masks and applies it to images.
    Highlights the intersection regions and overlays them on the original image.

    Args:
        images (list or np.ndarray): List or array of images. Each image should be a numpy array.
        shap_masks (list or np.ndarray): SHAP binary masks for important features.
        lime_masks (list or np.ndarray): LIME binary masks for important features.
        save (bool): Whether to save the images with important features.
        save_path (str): Directory path to save the images if save is True.

    Returns:
        intersection_images (list): List of images with intersection regions highlighted.
    """
    if len(images) != len(shap_masks) or len(images) != len(lime_masks):
        raise ValueError("The number of images, SHAP masks, and LIME masks must be the same.")
    
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)

    intersection_images = []

    for idx, (image, shap_mask, lime_mask) in enumerate(zip(images, shap_masks, lime_masks)):
        # Ensure image and masks have matching dimensions
        if image.shape[:2] != shap_mask.shape or image.shape[:2] != lime_mask.shape:
            raise ValueError(f"Image and masks at index {idx} have mismatched dimensions.")
        
        # Ensure masks are binary and of boolean type
        shap_mask = shap_mask.astype(bool)
        lime_mask = lime_mask.astype(bool)
        
        # Find the intersection of SHAP and LIME masks
        intersection_mask = np.logical_and(shap_mask, lime_mask)
        
        # Create a copy of the image to process
        intersection_image = np.copy(image)

        # Ensure the image is in RGB format for highlighting
        if intersection_image.ndim == 2:  # For grayscale images
            intersection_image = np.stack([intersection_image] * 3, axis=-1)  # Convert to RGB

        # Create an overlay image with the highlighted regions
        overlay_image = np.copy(intersection_image)
        highlight_color = [255, 0, 0]  # Red color

        # Apply the highlight color to the intersection areas
        overlay_image[intersection_mask] = highlight_color

        # Combine original image and overlay image
        final_image = np.where(intersection_mask[:, :, np.newaxis], overlay_image, intersection_image)

        intersection_images.append(final_image)
        
        if save:
            # Convert image to uint8 before saving
            image_to_save = img_as_ubyte(final_image)

            # Ensure image is either 2D or 3D
            if len(image_to_save.shape) not in [2, 3]:
                raise ValueError(f"Unexpected image shape for saving: {image_to_save.shape}")

            # Save the image
            imsave(os.path.join(save_path, f'intersection_image_{idx}.png'), image_to_save)
    
    return intersection_images


In [ ]:
intersection_images = find_intersection_and_save(test_features_[:300], masker_shap_[:300], train_labels_VGG19_lime, save=True, save_path="/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/Intersection/")

In [ ]:
### Train on new Data

In [ ]:
new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/intersection/'
new_prefix_ = 'ct_covid'

In [ ]:
def processImages(imgDirPath,binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_intersection_features_df =  processImages(new_path__,1)
ct_covid_intersection_features_df.shape

In [ ]:
cvd_imgs_intersection = [ct_noncovid_features_df, ct_covid_intersection_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs_intersection)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_intersection = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_intersection)

In [ ]:
input_data_intersection_x = cvd_imgs_dataset_intersection.iloc[:,:-1].to_numpy().reshape((2473,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_intersection['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_intersection['class_label'])
cvd_imgs_dataset_intersection
cvd_imgs_dataset_intersection = pd.get_dummies(cvd_imgs_dataset_intersection, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_intersection_y = np.array(cvd_imgs_dataset_intersection[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_intersection_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_intersection_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_intersection, test_features_intersection, train_labels_intersection, test_labels_intersection = train_test_split(
    input_data_intersection_x, output_label_intersection_y, test_size=.20, random_state=42)

In [ ]:
modelss = VGG19(input_shape=(300, 300, 1), classes=2)
modelss.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_vgg19.fit(train_features_intersection, train_labels_intersection, epochs=10, validation_data=(test_features_intersection, test_labels_intersection))

In [ ]:
###VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ZeroPadding2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def VGG16():
    model = Sequential()

    model.add(ZeroPadding2D((1,1), input_shape=(300, 300, 1)))  # Input shape corrected

    # Block 1
    model.add(Conv2D(2, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(2, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(4, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(4, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='inter'))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu', name='fc2'))
    model.add(Dense(2, activation='softmax'))  # Updated to 2 classes for binary classification

    return model



In [ ]:

modelss = VGG16()
modelss.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

modelss.fit(train_features, train_labels, epochs=10, batch_size=16)

In [ ]:
modelss.save('VGG16_model.h5')

In [ ]:
### Implementation of SHAP

In [ ]:
model_VGG16 = tf.keras.models.load_model('VGG16_model.h5')

In [ ]:
model_VGG16.summary()

In [ ]:
explainVGG16 = shap.GradientExplainer(model_VGG16,train_features)
explainVGG16 

In [ ]:
shap_valuesVGG16 = explainVGG16.shap_values(test_features)

In [ ]:
shap.image_plot(shap_valuesVGG16[:5], test_features[:5])

In [ ]:
my_new_shap_ = shap_valuesVGG16[:, :, :, :, 0]
my_new_shap_.shape

In [ ]:
threshold = np.percentile(np.abs(shap_values_), 95)
save_path = "dataset_important_features/VGG16/SHAP"  # Replace with your directory path
process_and_save_images(test_features, my_new_shap_, threshold, save_path)

In [ ]:
new_path = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/SHAP/'
new_prefix = 'ct_covid'

In [ ]:
ct_covid_shap_features_VGG16_df =  processImages(new_path,1)
ct_covid_shap_features_VGG16_df

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_shap_features_VGG16_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap_VGG16 = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap_VGG16)

In [ ]:
input_data_shap_VGG16_x = cvd_imgs_dataset_shap_VGG16.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap_VGG16['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap_VGG16['class_label'])
cvd_imgs_dataset_shap_VGG16
cvd_imgs_dataset_shap_VGG16 = pd.get_dummies(cvd_imgs_dataset_shap_VGG16, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_shap_VGG16_y = np.array(cvd_imgs_dataset_shap_VGG16[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_shap_VGG16_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_shap_VGG16_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap_VGG16, test_features_shap_VGG16, train_labels_shap_VGG16, test_labels_shap_VGG16 = train_test_split(
    input_data_shap_VGG16_x, output_label_shap_VGG16_y, test_size=.20, random_state=42)

In [ ]:
modelss = VGG16()
modelss.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

modelss.fit(train_features_shap_VGG16, train_labels_shap_VGG16, epochs=10, batch_size=16)

In [ ]:
### Applying LIME

In [ ]:
train_features[0].shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_VGG16.predict(images)

single_image = test_features[0] 
single_image.shape[0]
num_images_to_explain = 300
explainer = lime_image.LimeImageExplainer()
explanations = []
masks = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image = test_features[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    
    # Get explanation for the top predicted class
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks.append(mask)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
masks__ = np.array(masks)

In [ ]:
from skimage.transform import resize

def resize_mask(mask, image_shape):
    return resize(mask, image_shape[:2], mode='reflect', anti_aliasing=True, preserve_range=True)


In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image[0].shape}")
print(f"Original Mask shape: {masks[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask, image.shape) for mask, image in zip(masks, single_image)]

# Process images with the resized masks
processed_images = remove_non_important_features(
    images=single_image,
    masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/VGG16/LIME'  # Directory to save images
)


In [ ]:
### Train on new data

In [ ]:
new_path_ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/LIME/'
new_prefix_ = 'ct_covid'

In [ ]:
ct_covid_Lime_features_VGG19_df =  processImages(new_path,1)
ct_covid_Lime_features_VGG19_df

In [ ]:
input_data_lime_VGG16_x = cvd_imgs_dataset_lime_VGG16.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_lime_VGG16['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_lime_VGG16['class_label'])
cvd_imgs_dataset_lime_VGG16
cvd_imgs_dataset_lime_VGG16 = pd.get_dummies(cvd_imgs_dataset_lime_VGG16, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_lime_VGG16_y = np.array(cvd_imgs_dataset_lime_VGG16[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_lime_VGG16_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_lime_VGG16_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_VGG16_lime, test_features_VGG16_lime, train_labels_VGG16_lime, test_labels_VGG16_lime = train_test_split(
    input_data_lime_VGG16_x, output_label_lime_VGG16_y, test_size=.20, random_state=42)

In [ ]:
modelss = VGG16()
modelss.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

modelss.fit(train_features_VGG16_lime, train_labels_VGG16_lime, epochs=10, batch_size=16)

In [ ]:
#### Finding the Intersection

In [ ]:
masker_shap = np.array(masked_images_as_lists)

In [ ]:
masker_shap_ = masker_shap[:, 0, :, :]
len(masker_shap_[:300])

In [ ]:
test_features_ = test_features[:, :, :, 0]
len(test_features_[:300])

In [ ]:
intersection_images = find_intersection_and_save(test_features_[:300], masker_shap_[:300], train_labels_VGG19_lime, save=True, save_path="/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/Intersection/")

In [ ]:
#### Train on the intersection dataset

In [ ]:
new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/intersection/'
new_prefix_ = 'ct_covid'

In [ ]:
ct_covid_intersection_features_df =  processImages(new_path__,1)
ct_covid_intersection_features_df.shape

In [ ]:
cvd_imgs_intersection = [ct_noncovid_features_df, ct_covid_intersection_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs_intersection)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_intersection = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_intersection)

In [ ]:
input_data_intersection_x = cvd_imgs_dataset_intersection.iloc[:,:-1].to_numpy().reshape((2473,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_intersection['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_intersection['class_label'])
cvd_imgs_dataset_intersection
cvd_imgs_dataset_intersection = pd.get_dummies(cvd_imgs_dataset_intersection, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_intersection_y = np.array(cvd_imgs_dataset_intersection[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_intersection_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_intersection_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_intersection, test_features_intersection, train_labels_intersection, test_labels_intersection = train_test_split(
    input_data_intersection_x, output_label_intersection_y, test_size=.20, random_state=42)

In [ ]:
model_inter = VGG16()
model_inter.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model_inter.fit(train_features_intersection, train_labels_intersection, epochs=10, batch_size=16)

In [ ]:
### THIS IS FOR LIME ON OUR MODEL

In [ ]:
train_features[0].shape

In [ ]:
### Positive test

In [ ]:
test_features__[0].shape

In [ ]:
### Negative test

In [ ]:
test_features_[0].shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_mse.predict(images)

single_image = test_features[0] 
single_image.shape[0]
num_images_to_explain = 300
explainer = lime_image.LimeImageExplainer()
explanations = []
masks = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image = test_features[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    
    # Get explanation for the top predicted class
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks.append(mask)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
#### LIME for Positive test features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_mse.predict(images)

single_image__ = test_features__[0] 
single_image__.shape[0]
num_images_to_explain = 300
explainer__ = lime_image.LimeImageExplainer()
explanations__ = []
masks__ = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image__ = test_features__[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation__ = explainer__.explain_instance(single_image__,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations__.append(explanation__)
    
    # Get explanation for the top predicted class
    temp__, mask__ = explanation__.get_image_and_mask(explanation__.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks__.append(mask__)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features__[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image__.squeeze(), mask__), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
print(len(single_image__[1].shape), len(mask__[1].shape))  # Ensure they match in length


In [ ]:
import numpy as np
from skimage import data
from skimage.color import rgb2gray

# Sample images and masks (for demonstration)
# Replace these with actual images and masks from your dataset
#images = [data.astronaut(), data.coffee()]  # Example images
#masks = [np.random.randint(0, 2, size=image.shape[:2]) for image in images]  # Simulated LIME masks

resized_masks = [resize_mask(mask__, image.shape) for mask__, image in zip(masks__, single_image__)]

# Apply the function
processed_images = apply_lime_mask(
    images=single_image__,
    lime_masks=resized_masks,
    method='blur',           # Options: 'blur', 'mean', 'zero'
    blur_sigma=3,            # Adjust blur level if using the 'blur' method
    save=True,               # Set to True to save the images
    save_path='dataset_important_features/Lime/Positive'  # Directory to save images
)

# Display the result (optional)
import matplotlib.pyplot as plt

for i, img in enumerate(processed_images):
    plt.figure()
    plt.imshow(img)
    plt.title(f"Processed Image {i}")
    plt.axis('off')
plt.show()


In [ ]:
print(f"Original Image shape: {single_image__[0].shape}")
print(f"Original Mask shape: {resized_masks[0].shape}")


In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image__[0].shape}")
print(f"Original Mask shape: {masks__[0].shape}")

# Resize the mask if needed
#resized_masks = [resize_mask(mask__, image.shape) for mask__, image in zip(masks__, single_image__)]

# Process images with the resized masks
processed_images = apply_lime_mask(
    images=single_image__,
    lime_masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/Lime/Positive'  # Directory to save images
)


In [ ]:
#### LIME for Negative test features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_mse.predict(images)

single_image_ = test_features_[0] 
single_image_.shape[0]
num_images_to_explain = 300
explainer_ = lime_image.LimeImageExplainer()
explanations_ = []
masks_ = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image_ = test_features_[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation_ = explainer_.explain_instance(single_image_,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations_.append(explanation_)
    
    # Get explanation for the top predicted class
    temp_, mask_ = explanation_.get_image_and_mask(explanation_.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks_.append(mask_)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features_[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image_.squeeze(), mask_), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
print(f"Original Image shape: {single_image_[0].shape}")
print(f"Original Mask shape: {masks_[0].shape}")


In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image__[0].shape}")
print(f"Original Mask shape: {masks__[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask_, image.shape) for mask_, image in zip(masks_, single_image_)]

# Process images with the resized masks
processed_images = apply_lime_mask(
    images=single_image_,
    lime_masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/Lime/Negative'  # Directory to save images
)


In [ ]:
#### Retrain

In [ ]:
new_path___ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/Lime/Positive/'
new_prefix__ = 'ct_covid'

new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/Lime/Negative/'
new_prefix__ = 'non_ct_covid'

In [ ]:
ct_covid_features_df_lime =  processImages(new_path___,1)#1--> covid-19 positive
ct_noncovid_features_df_lime =  processImages(new_path__,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs_lime = [ct_noncovid_features_df_lime, ct_covid_features_df_lime]
cvd_imgs_dataset_lime = pd.concat(cvd_imgs_lime)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_lime_ = cvd_imgs_dataset_lime.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_lime_)

In [ ]:
cvd_imgs_dataset_lime_.shape

In [ ]:
input_data_lime_x = cvd_imgs_dataset_lime_.iloc[:,:-1].to_numpy().reshape((600,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_lime_['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_lime_['class_label'])
cvd_imgs_dataset_lime_
cvd_imgs_dataset_lime_ = pd.get_dummies(cvd_imgs_dataset_lime_, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_lime_y = np.array(cvd_imgs_dataset_lime_[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_lime_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_lime_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_lime, test_features_lime, train_labels_lime, test_labels_lime = train_test_split(
    input_data_lime_x, output_label_lime_y, test_size=.20, random_state=42)

In [ ]:
def CNN_Mod(model, num_hl, hl_list, hl_conv_activation, ol_activation, dropout_val, inputShape, filters_, kernel_size_, stride_poolSize):
    '''
    CNN_Mod(model, num_hl, hl_list, hl_conv_activation, ol_activation, dropout_val, inputShape, filters_, kernel_size_, stride_poolSize):
        model            = CNN Model
        num_hl           = Number of hidden layers
        hl_list          = List of hidden layer units
        hl_conv_activation = Activation function for convolutional layers
        ol_activation    = Activation function for the output layer
        dropout_val      = List of dropout values for hidden layers
        inputShape       = Shape of the input data (e.g., (300, 300, 1))
        filters_         = Number of filters for Conv layers
        kernel_size_     = Size of the kernel (height, width)
        stride_poolSize  = Stride and pool size (same value for both)
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    
    inputs = tf.keras.layers.Input(shape=inputShape)
    
    # Augment data (assuming data_augmentation is predefined)
    augmented = data_augmentation(inputs)
    
    # Convolutional Layer 1 with Max-Pooling
    conv_layer_1 = tf.keras.layers.Conv2D(filters=filters_, kernel_size=kernel_size_, 
                                          strides=stride_poolSize, activation=hl_conv_activation)(augmented)
    conv_layer_1_max_pool = tf.keras.layers.MaxPooling2D(pool_size=stride_poolSize)(conv_layer_1)
    x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    # Convolutional Layer 2 with Max-Pooling
    conv_layer_2 = tf.keras.layers.Conv2D(filters=filters_, kernel_size=kernel_size_, 
                                          strides=stride_poolSize, activation=hl_conv_activation)(x1)
    conv_layer_2_max_pool = tf.keras.layers.MaxPooling2D(pool_size=stride_poolSize)(conv_layer_2)
    x2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_2_max_pool)
    
    # Flattening the output of the last convolutional block
    flatten_layer = tf.keras.layers.Flatten()(x2)
    
    # Adding Dense and Dropout Layers
    dense_layers = flatten_layer
    for i in range(num_hl):
        dense_layers = tf.keras.layers.Dense(units=hl_list[i], activation=hl_conv_activation)(dense_layers)
        dense_layers = tf.keras.layers.Dropout(dropout_val[i])(dense_layers)
    
    # Output Layer
    logits = tf.keras.layers.Dense(units=2, activation=ol_activation)(dense_layers)
    
    # Create the Keras model
    model = tf.keras.Model(inputs=inputs, outputs=logits)
    
    return model


import tensorflow as tf

def compile_model(model, loss_, learningRate, metrics_):
    '''
    def compile_model(model, loss_, learningRate, metrics_):
    model: the model to compile
    loss_: the loss function
    learningRate: learning rate
    metrics_: list of metrics to track (e.g., ['accuracy'])
    '''
    # Use the legacy Adam optimizer if you're on an M1/M2 Mac
    optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate, weight_decay=0.001)

    # Alternatively, if needed, you can enable eager execution
    # tf.compat.v1.enable_eager_execution()

    model.compile(optimizer=optimizer, loss=loss_, metrics=metrics_)
    return model


def buildModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
  '''
  def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
  '''
  history = model.fit(
    train_features_lime, 
    train_labels_lime,
    validation_split = val_split_size,
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )
  
  return history

def evaluateModel_loss(history):
    print()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['Training loss','Validation loss'], loc = 'upper left')
    plt.savefig("Training_validation_loss_with_i.jpg")
    plt.show()
    plt.close()
    #plt.show()
    print()
def evaluateModel_accuracy(history):
    print()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Training and Validation Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epochs')
    plt.legend(['Training Accuracy','Validation Accuracy'], loc = 'upper left')
    #plt.show()
    plt.savefig("Training_validation_accuracy_with_i.jpg")
    plt.show()
    plt.close()
    print()

print(CNN_Mod.__doc__)

In [ ]:
image_size = 200
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Normalization(),
        tf.keras.layers.Resizing(image_size, image_size),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(factor=0.02),
        tf.keras.layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(train_features_lime)

In [ ]:
# initiallizing and building the model
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
cnn_1  = tf.keras.models.Sequential()
cnn_1  = CNN_Mod(cnn_1, 4, [288, 128, 33, 11],'relu','softmax',[0.00,0.35,0.00,0.00],(300,300,1),100,5,2)
#compile and run model here
cnn_1  = compile_model(cnn_1,'categorical_crossentropy',0.0015272304174499124,['accuracy'])
history = buildModel(cnn_1,0.20,128,100,10, "val_loss", "min")

In [ ]:
masks_ = np.array(masks)

In [ ]:
masks_.shape

In [ ]:
import numpy as np
from skimage.io import imsave
from skimage.util import img_as_ubyte
from skimage.filters import gaussian
import os

def apply_lime_mask(images, lime_masks, method='blur', blur_sigma=5, save=False, save_path='dataset_lime_filtered'):
    """
    Removes unimportant regions from images based on LIME masks.

    Parameters:
    - images (list or np.ndarray): List or array of original images.
    - lime_masks (list or np.ndarray): List or array of LIME masks (1 for important regions, 0 for unimportant).
    - method (str): Method to fill unimportant regions ('blur', 'mean', or 'zero').
    - blur_sigma (int): Sigma for Gaussian blur if 'blur' method is selected.
    - save (bool): Whether to save processed images.
    - save_path (str): Directory path to save images if save is True.

    Returns:
    - processed_images (list): List of images with unimportant regions removed.
    """
    
    # Verify input types
    if not isinstance(images, (list, np.ndarray)):
        raise TypeError("Images should be a list or numpy array of image arrays.")
        
    if not isinstance(lime_masks, (list, np.ndarray)):
        raise TypeError("LIME masks should be a list or numpy array of mask arrays.")
        
    if len(images) != len(lime_masks):
        raise ValueError("The number of images and LIME masks must be the same.")
    
    # Create save directory if needed
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)
    
    processed_images = []
    
    for idx, (image, lime_mask) in enumerate(zip(images, lime_masks)):
        # Ensure image and mask dimensions match
        if image.shape[:2] != lime_mask.shape:
            raise ValueError(f"Image and LIME mask at index {idx} have mismatched dimensions.")
        
        # Copy the image for processing
        processed_image = image.copy()
        
        # Apply the chosen method to unimportant regions (where lime_mask == 0)
        if method == 'blur':
            blurred_image = gaussian(processed_image, sigma=blur_sigma)
            processed_image[lime_mask == 0] = blurred_image[lime_mask == 0]
        
        elif method == 'mean':
            mean_value = image.mean(axis=(0, 1), keepdims=True) if image.ndim == 3 else image.mean()
            processed_image[lime_mask == 0] = mean_value
        
        elif method == 'zero':
            processed_image[lime_mask == 0] = 0
        
        else:
            raise ValueError("Invalid method chosen. Available methods: 'blur', 'mean', 'zero'.")
        
        # Append processed image to the list
        processed_images.append(processed_image)
        
        # Save processed image if required
        if save:
            image_to_save = img_as_ubyte(processed_image)
            imsave(os.path.join(save_path, f'lime_filtered_image_{idx}.png'), image_to_save, check_contrast=False)
            print(f"Saved: {os.path.join(save_path, f'lime_filtered_image_{idx}.png')}")
    
    return processed_images


In [ ]:
import numpy as np
from lime import lime_image
from skimage.segmentation import felzenszwalb


def predict_fn(images):
    return model_mse.predict(images)

num_images_to_explain = 300
# Initialize LIME explainer
explainer = lime_image.LimeImageExplainer()

# Generate explanations and masks for each image
masks = []
for i in range(num_images_to_explain):
    single_image = test_features[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    # Get mask for the top predicted label
    _, mask = explanation.get_image_and_mask(
        label=explanation.top_labels[0],
        positive_only=True,
        hide_rest=False,
        num_features=5
    )
    masks.append(mask)


In [ ]:
print(len(single_image), len(mask))  # Ensure they match in length


In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
plt.title(f"Explanation for Image {i+1}")
plt.axis('off')
plt.show()

In [ ]:
from skimage.transform import resize

def resize_mask(mask, image_shape):
    return resize(mask, image_shape[:2], mode='reflect', anti_aliasing=True, preserve_range=True)


In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image[0].shape}")
print(f"Original Mask shape: {masks[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask, image.shape) for mask, image in zip(masks, single_image)]

# Process images with the resized masks
processed_images = remove_non_important_features(
    images=single_image,
    masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/Lime'  # Directory to save images
)


In [ ]:
lime_test_feature = np.array(processed_images)
lime_test_feature.shape

In [ ]:
for i in range(5):
    imshow(lime_test_feature[i])
    show()
    

In [ ]:
##### Train the new dataset on the model

In [ ]:
new_path_ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/Lime/'
new_prefix_ = 'ct_covid'


In [ ]:
def processImages(imgDirPath,binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_lime_features_df =  processImages(new_path_,1)
ct_covid_lime_features_df.shape

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_lime_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_lime = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_lime)

In [ ]:
cvd_imgs_dataset_lime.shape

In [ ]:
input_data_lime_x = cvd_imgs_dataset_lime.iloc[:,:-1].to_numpy().reshape((2473,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_lime['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_lime['class_label'])
cvd_imgs_dataset_lime
cvd_imgs_dataset_lime = pd.get_dummies(cvd_imgs_dataset_lime, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_lime_y = np.array(cvd_imgs_dataset_lime[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_lime_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_lime_y.shape))

In [ ]:
for i in range(3):
    imshow(input_data_lime_x[i])
    show()

In [ ]:
from sklearn.model_selection import train_test_split

train_features_lime, test_features_lime, train_labels_lime, test_labels_lime = train_test_split(
    input_data_lime_x, output_label_lime_y, test_size=.20, random_state=42)

In [ ]:
train_features_lime.shape

In [ ]:
train_labels_lime.shape

In [ ]:
# image_size = 200
# data_augmentation = keras.Sequential(
#     [
#         tf.keras.layers.Normalization(),
#         tf.keras.layers.Resizing(image_size, image_size),
#         tf.keras.layers.RandomFlip("horizontal"),
#         tf.keras.layers.RandomRotation(factor=0.02),
#         tf.keras.layers.RandomZoom(
#             height_factor=0.2, width_factor=0.2
#         ),
#     ],
#     name="data_augmentation",
# )
# # Compute the mean and the variance of the training data for normalization.
# data_augmentation.layers[0].adapt(train_features_shap)

In [ ]:
def CNN_Mod(model, num_hl, hl_list, hl_conv_activation, ol_activation, dropout_val, inputShape, filters_, kernel_size_, stride_poolSize):
    '''
    CNN_Mod(model, num_hl, hl_list, hl_conv_activation, ol_activation, dropout_val, inputShape, filters_, kernel_size_, stride_poolSize):
        model            = CNN Model
        num_hl           = Number of hidden layers
        hl_list          = List of hidden layer units
        hl_conv_activation = Activation function for convolutional layers
        ol_activation    = Activation function for the output layer
        dropout_val      = List of dropout values for hidden layers
        inputShape       = Shape of the input data (e.g., (300, 300, 1))
        filters_         = Number of filters for Conv layers
        kernel_size_     = Size of the kernel (height, width)
        stride_poolSize  = Stride and pool size (same value for both)
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    
    inputs = tf.keras.layers.Input(shape=inputShape)
    
    # Augment data (assuming data_augmentation is predefined)
    #augmented = data_augmentation(inputs)
    
    # Convolutional Layer 1 with Max-Pooling
    conv_layer_1 = tf.keras.layers.Conv2D(filters=filters_, kernel_size=kernel_size_, 
                                          strides=stride_poolSize, activation=hl_conv_activation)(inputs)
    conv_layer_1_max_pool = tf.keras.layers.MaxPooling2D(pool_size=stride_poolSize)(conv_layer_1)
    x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    # Convolutional Layer 2 with Max-Pooling
    conv_layer_2 = tf.keras.layers.Conv2D(filters=filters_, kernel_size=kernel_size_, 
                                          strides=stride_poolSize, activation=hl_conv_activation)(x1)
    conv_layer_2_max_pool = tf.keras.layers.MaxPooling2D(pool_size=stride_poolSize)(conv_layer_2)
    x2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_2_max_pool)
    
    # Flattening the output of the last convolutional block
    flatten_layer = tf.keras.layers.Flatten()(x2)
    
    # Adding Dense and Dropout Layers
    dense_layers = flatten_layer
    for i in range(num_hl):
        dense_layers = tf.keras.layers.Dense(units=hl_list[i], activation=hl_conv_activation)(dense_layers)
        dense_layers = tf.keras.layers.Dropout(dropout_val[i])(dense_layers)
    
    # Output Layer
    logits = tf.keras.layers.Dense(units=2, activation=ol_activation)(dense_layers)
    
    # Create the Keras model
    model = tf.keras.Model(inputs=inputs, outputs=logits)
    
    return model


import tensorflow as tf

def compile_model(model, loss_, learningRate, metrics_):
    '''
    def compile_model(model, loss_, learningRate, metrics_):
    model: the model to compile
    loss_: the loss function
    learningRate: learning rate
    metrics_: list of metrics to track (e.g., ['accuracy'])
    '''
    # Use the legacy Adam optimizer if you're on an M1/M2 Mac
    optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate, weight_decay=0.001)

    # Alternatively, if needed, you can enable eager execution
    # tf.compat.v1.enable_eager_execution()

    model.compile(optimizer=optimizer, loss=loss_, metrics=metrics_)
    return model


def buildModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
  '''
  def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
  '''
  history = model.fit(
    train_features_lime, 
    train_labels_lime,
    validation_split = val_split_size,
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )
  
  return history

def evaluateModel_loss(history):
    print()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['Training loss','Validation loss'], loc = 'upper left')
    plt.savefig("Training_validation_loss_with_i.jpg")
    plt.show()
    plt.close()
    #plt.show()
    print()
def evaluateModel_accuracy(history):
    print()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Training and Validation Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epochs')
    plt.legend(['Training Accuracy','Validation Accuracy'], loc = 'upper left')
    #plt.show()
    plt.savefig("Training_validation_accuracy_with_i.jpg")
    plt.show()
    plt.close()
    print()

print(CNN_Mod.__doc__)

In [ ]:
# initiallizing and building the model
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
cnn_1  = tf.keras.models.Sequential()
cnn_1  = CNN_Mod(cnn_1, 4, [288, 128, 33, 11],'relu','softmax',[0.00,0.35,0.00,0.00],(300,300,1),100,5,2)
#compile and run model here
cnn_1  = compile_model(cnn_1,'categorical_crossentropy',0.0015272304174499124,['accuracy'])
history = buildModel(cnn_1,0.20,128,100,10, "val_loss", "min")

In [ ]:
evaluateModel_loss(history)

In [ ]:
evaluateModel_accuracy(history)

In [ ]:
### Find the intersection between LIME and SHAP

In [ ]:
print(dir(masker))


In [ ]:
masker_shap = np.array(masked_images_as_lists)

In [ ]:
masker_shap.shape

In [ ]:
masker_shap_ = masker_shap[:, 0, :, :]
len(masker_shap_[:300])

In [ ]:
test_features.shape

In [ ]:
test_features_ = test_features[:, :, :, 0]
len(test_features_[:300])

In [ ]:
lime_masks = masks_
(lime_masks.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def intersection_shap_lime(images, shap_masks, lime_masks, method='intersection', save=False, save_path='shap_lime_intersection'):
    """
    Finds and saves the intersection between SHAP and LIME masks for a dataset of images.

    Parameters:
    - images (list or np.ndarray): List or array of original images.
    - shap_masks (list or np.ndarray): List or array of SHAP masks (1 for important, 0 for unimportant).
    - lime_masks (list or np.ndarray): List or array of LIME masks (1 for important, 0 for unimportant).
    - method (str): Method to highlight intersection ('intersection', 'shap_only', 'lime_only').
    - save (bool): Whether to save the images showing the intersection.
    - save_path (str): Directory to save images if save is True.

    Returns:
    - intersected_images (list): List of images with intersected regions highlighted.
    """
    if len(images) != len(shap_masks) or len(images) != len(lime_masks):
        raise ValueError("The number of images, SHAP masks, and LIME masks must be the same.")

    # Create save directory if needed
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)

    intersected_images = []

    for idx, (image, shap_mask, lime_mask) in enumerate(zip(images, shap_masks, lime_masks)):
        # Check if the mask dimensions match the image
        if image.shape[:2] != shap_mask.shape or image.shape[:2] != lime_mask.shape:
            raise ValueError(f"Image and mask dimensions mismatch at index {idx}.")

        # Intersection of SHAP and LIME masks
        if method == 'intersection':
            intersect_mask = np.logical_and(shap_mask, lime_mask)
        elif method == 'shap_only':
            intersect_mask = np.logical_and(shap_mask, np.logical_not(lime_mask))
        elif method == 'lime_only':
            intersect_mask = np.logical_and(lime_mask, np.logical_not(shap_mask))
        else:
            raise ValueError("Invalid method. Available methods: 'intersection', 'shap_only', 'lime_only'.")

        # Create an overlay on the image to highlight intersected regions
        intersected_image = image.copy()
        intersected_image[intersect_mask == 0] = 0  # Set non-intersected regions to black

        intersected_images.append(intersected_image)

        # Save intersected image if required
        if save:
            plt.imsave(f"{save_path}/intersected_image_{idx}.png", intersected_image)
            print(f"Saved intersected image at: {save_path}/intersected_image_{idx}.png")

    return intersected_images



In [ ]:
# Sample Usage
# Assume images, shap_masks, and lime_masks are already loaded and are lists of images/masks
# Each mask should have values of 1 for important and 0 for unimportant regions.

images = [np.random.rand(64, 64, 3) for _ in range(5)]  # Example dataset
shap_masks = [np.random.randint(0, 2, size=(64, 64)) for _ in range(5)]  # Simulated SHAP masks
lime_masks = [np.random.randint(0, 2, size=(64, 64)) for _ in range(5)]  # Simulated LIME masks

# Find the intersection
intersected_images = intersection_shap_lime(images, shap_masks, lime_masks, method='intersection', save=True, save_path='shap_lime_intersection')

# Display results
for i, img in enumerate(intersected_images):
    plt.figure()
    plt.imshow(img)
    plt.title(f"Intersected Image {i}")
    plt.axis('off')
plt.show()


In [ ]:
intersection_images = intersection_shap_lime(test_features_[:300], masker_shap_[:300], lime_masks, method='intersection', save=True, save_path="/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/intersection/")

In [ ]:
##### Train the new dataset on the model

In [ ]:
new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/intersection/'
new_prefix_ = 'ct_covid'

In [ ]:
def processImages(imgDirPath,binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_intersection_features_df =  processImages(new_path__,1)
ct_covid_intersection_features_df.shape

In [ ]:
cvd_imgs_intersection = [ct_noncovid_features_df, ct_covid_intersection_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs_intersection)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_intersection = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_intersection)

In [ ]:
cvd_imgs_dataset_intersection.shape

In [ ]:
input_data_intersection_x = cvd_imgs_dataset_intersection.iloc[:,:-1].to_numpy().reshape((2473,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_intersection['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_intersection['class_label'])
cvd_imgs_dataset_intersection
cvd_imgs_dataset_intersection = pd.get_dummies(cvd_imgs_dataset_intersection, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_intersection_y = np.array(cvd_imgs_dataset_intersection[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_intersection_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_intersection_y.shape))

In [ ]:
for i in range(3):
    imshow(input_data_intersection_x[i])
    show()

In [ ]:
from sklearn.model_selection import train_test_split

train_features_intersection, test_features_intersection, train_labels_intersection, test_labels_intersection = train_test_split(
    input_data_intersection_x, output_label_intersection_y, test_size=.20, random_state=42)

In [ ]:
train_features_intersection.shape

In [ ]:
train_labels_intersection.shape

In [ ]:
def CNN_Mod(model, num_hl, hl_list, hl_conv_activation, ol_activation, dropout_val, inputShape, filters_, kernel_size_, stride_poolSize):
    '''
    CNN_Mod(model, num_hl, hl_list, hl_conv_activation, ol_activation, dropout_val, inputShape, filters_, kernel_size_, stride_poolSize):
        model            = CNN Model
        num_hl           = Number of hidden layers
        hl_list          = List of hidden layer units
        hl_conv_activation = Activation function for convolutional layers
        ol_activation    = Activation function for the output layer
        dropout_val      = List of dropout values for hidden layers
        inputShape       = Shape of the input data (e.g., (300, 300, 1))
        filters_         = Number of filters for Conv layers
        kernel_size_     = Size of the kernel (height, width)
        stride_poolSize  = Stride and pool size (same value for both)
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    
    inputs = tf.keras.layers.Input(shape=inputShape)
    
    # Augment data (assuming data_augmentation is predefined)
    augmented = data_augmentation(inputs)
    
    # Convolutional Layer 1 with Max-Pooling
    conv_layer_1 = tf.keras.layers.Conv2D(filters=filters_, kernel_size=kernel_size_, 
                                          strides=stride_poolSize, activation=hl_conv_activation)(inputs)
    conv_layer_1_max_pool = tf.keras.layers.MaxPooling2D(pool_size=stride_poolSize)(conv_layer_1)
    x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    # Convolutional Layer 2 with Max-Pooling
    conv_layer_2 = tf.keras.layers.Conv2D(filters=filters_, kernel_size=kernel_size_, 
                                          strides=stride_poolSize, activation=hl_conv_activation)(x1)
    conv_layer_2_max_pool = tf.keras.layers.MaxPooling2D(pool_size=stride_poolSize)(conv_layer_2)
    x2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_2_max_pool)
    
    # Flattening the output of the last convolutional block
    flatten_layer = tf.keras.layers.Flatten()(x2)
    
    # Adding Dense and Dropout Layers
    dense_layers = flatten_layer
    for i in range(num_hl):
        dense_layers = tf.keras.layers.Dense(units=hl_list[i], activation=hl_conv_activation)(dense_layers)
        dense_layers = tf.keras.layers.Dropout(dropout_val[i])(dense_layers)
    
    # Output Layer
    logits = tf.keras.layers.Dense(units=2, activation=ol_activation)(dense_layers)
    
    # Create the Keras model
    model = tf.keras.Model(inputs=inputs, outputs=logits)
    
    return model


import tensorflow as tf

def compile_model(model, loss_, learningRate, metrics_):
    '''
    def compile_model(model, loss_, learningRate, metrics_):
    model: the model to compile
    loss_: the loss function
    learningRate: learning rate
    metrics_: list of metrics to track (e.g., ['accuracy'])
    '''
    # Use the legacy Adam optimizer if you're on an M1/M2 Mac
    optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate, weight_decay=0.001)

    # Alternatively, if needed, you can enable eager execution
    # tf.compat.v1.enable_eager_execution()

    model.compile(optimizer=optimizer, loss=loss_, metrics=metrics_)
    return model


def buildModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
  '''
  def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
  '''
  history = model.fit(
    train_features_intersection, 
    train_labels_intersection,
    validation_split = val_split_size,
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )
  
  return history

def evaluateModel_loss(history):
    print()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['Training loss','Validation loss'], loc = 'upper left')
    plt.savefig("Training_validation_loss_with_i.jpg")
    plt.show()
    plt.close()
    #plt.show()
    print()
def evaluateModel_accuracy(history):
    print()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Training and Validation Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epochs')
    plt.legend(['Training Accuracy','Validation Accuracy'], loc = 'upper left')
    #plt.show()
    plt.savefig("Training_validation_accuracy_with_i.jpg")
    plt.show()
    plt.close()
    print()

print(CNN_Mod.__doc__)

In [ ]:
# initiallizing and building the model
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
cnn_1  = tf.keras.models.Sequential()
cnn_1  = CNN_Mod(cnn_1, 4, [288, 128, 33, 11],'relu','softmax',[0.00,0.35,0.00,0.00],(300,300,1),100,5,2)
#compile and run model here
cnn_1  = compile_model(cnn_1,'categorical_crossentropy',0.0015272304174499124,['accuracy'])
history = buildModel(cnn_1,0.20,128,100,10, "val_loss", "min")

In [ ]:
evaluateModel_loss(history)

In [ ]:
evaluateModel_accuracy(history)

### Data Augmentation

In [ ]:
image_size = 200
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Normalization(),
        tf.keras.layers.Resizing(image_size, image_size),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(factor=0.02),
        tf.keras.layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(train_features)

### Building Model

In [ ]:
def CNN_Mod(model, num_hl, hl_list,hl_conv_activation, ol_activation, dropout_val, inputShape,filters_,kernel_size_,stride_poolSize):
    '''
    CNN_Mod(model, num_hl, hl_list,hl_conv_activation, ol_activation, dropout_val, inputShape,filters_,kernel_size_,stride_poolSize):
        model = CNN Model
        num_hl = number of hidden layers
        hl_list = list of hidden layers
        hl_activation = hidden layer activation function
        out_activation = output layer activation function
        dropout_val = Dropout value
        inputShape = shape of input layer
        filters_ = filter size of Conv layer
        kernel_size_ = size of kernel (x,x)
        strides_poolSize = strides and max_pool_szie
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    inputs = tf.keras.layers.Input(shape=inputShape)
    # Augment data.
    augmented = data_augmentation(inputs)
    
    #connnnvolultion layer 1 with Map-pooling
    conv_layer_1 = tf.keras.layers.Conv2D(filters = filters_,kernel_size = kernel_size_,strides = stride_poolSize, activation = hl_conv_activation)(augmented)
    conv_layer_1_max_pool = tf.keras.layers.MaxPool2D(strides=stride_poolSize)(conv_layer_1)
    x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    #connnnvolultion layer 1 with Map-pooling
    conv_layer_2 = tf.keras.layers.Conv2D(filters = filters_,kernel_size = kernel_size_,strides = stride_poolSize, activation = hl_conv_activation)(x1)
    conv_layer_2_max_pool = tf.keras.layers.MaxPool2D(strides=stride_poolSize)(conv_layer_2)
    x2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    flatten_layer = tf.keras.layers.Flatten()(x2)
    
    dense_layers = tf.keras.layers.Dense(units = hl_list[0], activation = hl_conv_activation)(flatten_layer)
    dense_layers = tf.keras.layers.Dropout(dropout_val[0])(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hl_list[1], activation = hl_conv_activation)(dense_layers)
    dense_layers = tf.keras.layers.Dropout(dropout_val[1])(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hl_list[2], activation = hl_conv_activation)(dense_layers)
    dense_layers = tf.keras.layers.Dropout(dropout_val[2])(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hl_list[3], activation = hl_conv_activation)(dense_layers)
    dense_layers = tf.keras.layers.Dropout(dropout_val[3])(dense_layers)
    
    logits = tf.keras.layers.Dense(units = 2, activation = ol_activation)(dense_layers)
    
    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs,outputs=logits)
    
    return model


def compile_model(model, loss_, learningRate, metrics_):
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest [mse', 'mae'] #since a regression model
    '''
    model.compile(tf.keras.optimizers.Adam(learning_rate=learningRate,weight_decay=0.001),loss=loss_,metrics=metrics_)
    return model

def buildModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
  '''
  def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
  '''
  history = model.fit(
    train_features, 
    train_labels,
    validation_split = val_split_size,
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )
  
  return history

def evaluateModel_loss(history):
    print()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['Training loss','Validation loss'], loc = 'upper left')
    plt.savefig("Training_validation_loss_with_i.jpg")
    plt.show()
    plt.close()
    #plt.show()
    print()
def evaluateModel_accuracy(history):
    print()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Training and Validation Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epochs')
    plt.legend(['Training Accuracy','Validation Accuracy'], loc = 'upper left')
    #plt.show()
    plt.savefig("Training_validation_accuracy_with_i.jpg")
    plt.show()
    plt.close()
    print()

print(CNN_Mod.__doc__)

In [ ]:
# initiallizing and building the model
#cnn_1  = tf.keras.models.Sequential()

In [ ]:
'''
Hyperparameters:
Hyperparameters:
Convolution Layers filter Count: : 100
Convolution Layers Kernel Size: : 5
Max-pooling_Stride_ Size: : 2
Hiddel Layer 1 Units: 2884
dropout1: False
Hiddel Layer 2 Units: 1288
dropout2: True
Hiddel Layer 3 Units: 332
dropout3: False
Hiddel Layer 4 Units: 116
dropout4: False
lr: 0.0015272304174499124
Score: 0.9326164722442627
'''

In [ ]:
# initiallizing and building the model
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
cnn_1  = tf.keras.models.Sequential()
cnn_1  = CNN_Mod(cnn_1, 4, [288, 128, 33, 11],'relu','softmax',[0.00,0.35,0.00,0.00],(300,300,1),100,5,2)
#compile and run model here
cnn_1  = compile_model(cnn_1,'categorical_crossentropy',0.0015272304174499124,['accuracy'])
history = buildModel(cnn_1,0.20,128,100,10, "val_loss", "min")

In [ ]:
evaluateModel_loss(history)

In [ ]:
evaluateModel_accuracy(history)

In [ ]:
cnn_1.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(cnn_1, to_file='cnn_1_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
cnn_1.save('cvd_cnn_1.h5')

In [ ]:
cnn_1.save_weights('cvd_cnn_weight_1.h5')

#### Testing Model

In [ ]:
label_predicted = cnn_1.predict((test_features))

In [ ]:
n_samples = 10

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    sample_image = test_features[index].reshape(300, 300)
    plt.imshow(sample_image, cmap="binary")
    plt.title("Label:" + str(test_labels[index].argmax()))
    plt.axis("off")

plt.show()

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    #sample_image = image_predicted[index].reshape(300, 300)
    #plt.imshow(sample_image, cmap="binary")
    plt.title("Predicted:" + str(np.argmax(label_predicted[index])))
    plt.axis("off")

plt.show()

In [ ]:
def getCorrectLabel(x):
        if x == 0:
            return 'Non-Covid'
        if x == 1:
            return 'Covid'
        
def D1_to_D2(y_pred):
    y_true = {'y_true':[i[0] for i in y_pred]}
    y_true_df = pd.DataFrame(y_true)
    y_true_df = pd.get_dummies(y_true_df, columns =['y_true'])
    output_label_y = np.array(y_true_df[y_true_df.columns])
    return output_label_y

def getModelEvaluation(model, test_features_,test_labels_):
    predicted_labels = model.predict(np.stack(test_features_))
    df = pd.DataFrame(predicted_labels)
    df['Predicted_Labels'] = np.array(df.iloc[:,:2]).argmax(axis =1)
    df['Predicted_Labels'] = df['Predicted_Labels'].apply(getCorrectLabel)
    if (test_labels_.shape[1]==2):
        df['Actual_Labels'] = test_labels_[:,:2].argmax(axis = 1)
    else:
        df['Actual_Labels'] = D1_to_D2(test_labels_)[:,:2].argmax(axis = 1)
    df['Actual_Labels'] = df['Actual_Labels'].apply(getCorrectLabel)
    # df['Probality'] = np.max(np.array(df.iloc[:,:7]))
    #df.drop([0,1,2,3,4,5,6,7,8,9],axis =1,inplace=True)
    #print(df.head(100))
    cm = pd.crosstab(df.Predicted_Labels, df.Actual_Labels)
    fig = plt.figure(figsize=(20,5))
    ax1 = plt.subplot(121)
    sns.heatmap(cm,annot = True,cmap='Blues')
    ax1.set_title('')
    # Saving the figure.
    plt.savefig("test_Confusion_Matrix_with_i.jpg")
    plt.show()
    plt.close()

    pred = model.evaluate(test_features_,test_labels_)
    print("loss = " + str(pred[0]))
    print("test accuracy = " + str(pred[1]))

In [ ]:
import seaborn as sns
getModelEvaluation(cnn_1,test_features,test_labels)

### Testing the Trained Model on a different Dataset
- https://www.kaggle.com/datasets/luisblanche/covidct

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))

path1 = '/home/rkannan/Desktop/Cvd19_Classification/dataset/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/rkannan/Desktop/Cvd19_Classification/dataset/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

def processImage(imgDirPath,binary_label):
    img_names = list()
    with os.scandir(imgDirPath) as dirs:
        for entry in dirs:
            img_names.append(entry.name)
    #Creating features for images
    all_features = []
    for img in img_names:
        path = imgDirPath + img
        cv_img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)
        #create features for machine learning
        nFeatures = (cv_img2.shape[0]*cv_img2.shape[1])
        features = np.reshape(cv_img2, nFeatures)
        all_features.append(features)
        #print(features.shape)
    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]),dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]),dtype=int)
    return imgs_df


ct_covid_features_df =  processImage(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImage(path2,0)#0 ---> covnid-19 negative
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
display(cvd_imgs_dataset)

input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((746,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset_ = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset_[['output_encode_0','output_encode_1']])
#output_label_y = np.array(cvd_imgs_dataset[['output_encode']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))
#print('Output_y Data Shape for evaluation: \n{0}'.format(output_label_y_eval.shape))

In [ ]:
y_test_new = output_label_y
x_test_new = input_data_x

### Loading Trained Model on old dataset

In [ ]:
cnn_old  = tf.keras.models.Sequential()
cnn_old  = CNN_Mod(cnn_old, 4, [288, 128, 33, 11],'relu','softmax',[0.00,0.35,0.00,0.00],(300,300,1),100,5,2)
#compile and run model here
cnn_old  = compile_model(cnn_old,'categorical_crossentropy',0.0015272304174499124,['accuracy'])
cnn_old.load_weights('cvd_cnn_weight_1.h5')

In [ ]:
getModelEvaluation(cnn_old,x_test_new,y_test_new)

## Hyper Parameter Tuning

In [ ]:
import keras_tuner as kt
def KT_hp_build(hp,shape = (300,300,1), dropout_rate = 0.35):
    #initiallizing the model
    cnn = tf.keras.models.Sequential()
    
    filters_ = hp.Int("Convolution Layers filter Count: ",min_value = 100, max_value = 130, step = 16 )
    kernel_size_ = hp.Int("Convolution Layers Kernel Size: ",min_value = 5, max_value = 10 ,step = 3)
    strides_poolSize = hp.Choice("Max-pooling_Stride_ Size: ",[2])
    hl_conv_activation = 'relu'
    ol_activation = 'softmax'
    
    inputs = tf.keras.layers.Input(shape=shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    
    #connnnvolultion layer 1 with Map-pooling
    conv_layer_1 = tf.keras.layers.Conv2D(filters = filters_,kernel_size = kernel_size_,strides = stride_poolSize, activation = hl_conv_activation)(augmented)
    conv_layer_1_max_pool = tf.keras.layers.MaxPool2D(strides=stride_poolSize)(conv_layer_1)
    x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    #connnnvolultion layer 1 with Map-pooling
    conv_layer_2 = tf.keras.layers.Conv2D(filters = filters_,kernel_size = kernel_size_,strides = stride_poolSize, activation = hl_conv_activation)(x1)
    conv_layer_2_max_pool = tf.keras.layers.MaxPool2D(strides=stride_poolSize)(conv_layer_2)
    x2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    flatten_layer = tf.keras.layers.Flatten()(x2)
    
    dense_layers = tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 1 Units",min_value=2500, max_value=3000,step = 16), activation = hl_conv_activation)(flatten_layer)
    if hp.Boolean("dropout1"):
        dense_layers = tf.keras.layers.Dropout(dropout_rate)(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 2 Units",min_value=1000, max_value=1500,step = 32), activation = hl_conv_activation)(dense_layers)
    if hp.Boolean("dropout2"):
        dense_layers = tf.keras.layers.Dropout(dropout_rate)(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 3 Units",min_value=300, max_value=500,step = 32), activation = hl_conv_activation)(dense_layers)
    if hp.Boolean("dropout3"):
        dense_layers = tf.keras.layers.Dropout(dropout_rate)(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 4 Units",min_value=100, max_value=150,step = 4), activation = hl_conv_activation)(dense_layers)
    if hp.Boolean("dropout4"):
        dense_layers = tf.keras.layers.Dropout(dropout_rate)(dense_layers)
    
    logits = tf.keras.layers.Dense(units = 2, activation = ol_activation)(dense_layers)
    
    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs,outputs=logits)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=0.001, max_value=0.002, sampling="log")
    
    cnn.compile(
      #optimizer = 'adam', 
      optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
      loss = 'categorical_crossentropy', 
      metrics = ['accuracy'])
    return model
    
#     # First Convolution Layer with Max-Pooling
#     cnn.add(tf.keras.layers.Conv2D(filters = filters_val,kernel_size = kernel_size_val,strides = strides_poolSize, activation ='relu',input_shape = shape))
#     cnn.add(tf.keras.layers.MaxPool2D(pool_size=strides_poolSize, strides=strides_poolSize))
    
#     # second Convolution Layer with Max-Pooling
#     cnn.add(tf.keras.layers.Conv2D(filters = filters_val,kernel_size = kernel_size_val,strides = strides_poolSize, activation ='relu'))
#     cnn.add(tf.keras.layers.MaxPool2D(pool_size=strides_poolSize, strides=strides_poolSize))
    
#     # Third Convolution Layer with Max-Pooling
#     cnn.add(tf.keras.layers.Conv2D(filters = filters_val,kernel_size = kernel_size_val,strides = strides_poolSize, activation ='relu'))
#     cnn.add(tf.keras.layers.MaxPool2D(pool_size=strides_poolSize, strides=strides_poolSize))
    
#     #Flattening
#     cnn.add(tf.keras.layers.Flatten())
    
#     #Full Connection
#     cnn.add(tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 1 Units",min_value=2500, max_value=3000,step = 16), activation = hl_conv_activation))
#     if hp.Boolean("dropout1"):
#         cnn.add(tf.keras.layers.Dropout(dropout_rate))

#     cnn.add(tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 2 Units",min_value=1000, max_value=1500,step = 32), activation = hl_conv_activation))
#     if hp.Boolean("dropout2"):
#         cnn.add(tf.keras.layers.Dropout(dropout_rate))

#     cnn.add(tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 3 Units",min_value=300, max_value=500,step = 32), activation = hl_conv_activation))
#     if hp.Boolean("dropout3"):
#         cnn.add(tf.keras.layers.Dropout(dropout_rate))
    
#     cnn.add(tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 4 Units",min_value=100, max_value=150,step = 4), activation = hl_conv_activation))
#     if hp.Boolean("dropout4"):
#         cnn.add(tf.keras.layers.Dropout(dropout_rate))
    
#     #output
#     cnn.add(tf.keras.layers.Dense(units = 2, activation = ol_activation))
    
    
#     # Define the optimizer learning rate as a hyperparameter.
#     learning_rate = hp.Float("lr", min_value=0.001, max_value=0.002, sampling="log")
    
#     cnn.compile(
#       #optimizer = 'adam', 
#       optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
#       loss = 'categorical_crossentropy', 
#       metrics = ['accuracy'])
#     return cnn

In [ ]:
KT_hp_build(kt.HyperParameters())

In [ ]:
#initialize the tuner
tuner = kt.RandomSearch(
    hypermodel= KT_hp_build,
    objective="val_accuracy", 
    overwrite = True,# Do not resume the previous search in the same directory.
    max_trials= 5,
    directory = "models/param_tuning",  # Set a directory to store the intermediate results.
    project_name= "param_tuning"
)

In [ ]:
tuner.search_space_summary() #print a summary of the search space

In [ ]:
#Visuallizing the hyperparameter tuning process
tuner.search(train_features, train_labels, 
             validation_data=(test_features, test_labels),
             validation_split=0.20,
             batch_size = 128,
             epochs = 100,
             # Use the TensorBoard callback.
             # The logs will be write to "models/tb_logs".
             callbacks=[
                tf.keras.callbacks.EarlyStopping(monitor="loss",patience=10,verbose=1,mode="min")]
                #tf.keras.callbacks.TensorBoard("models/tb_logs")],
             )

In [ ]:
tuner.results_summary()